In [19]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Hyper-parameters 
input_size = 784 #  MNIST image is 28x28
hidden_size = 100 
num_classes = 10 # number image 0-9
num_epochs = 10
batch_size = 100
learning_rate = 0.001

# use cpu device = torch.device('cpu')


In [20]:

#MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

examples = iter(test_loader)


# Check out the data
## samples, labels = next(examples)
## print(samples.shape, labels.shape)

example_data, example_targets = next(examples)

## for i in range(9):
##   plt.subplot(3,3, i+1)
##   plt.imshow(samples[i][0]) 

In [21]:
# feed forward neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        # Inputs to hidden layer linear transformation
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        # no activation and no softmax at the end
        return out



In [22]:
# create model
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

# Train the model
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # origin shape: [100, 1, 28, 28]
        # resized: [100, 784]
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 600 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')



Epoch [1/10], Step [600/600], Loss: 0.1452
Epoch [2/10], Step [600/600], Loss: 0.1741
Epoch [3/10], Step [600/600], Loss: 0.1414
Epoch [4/10], Step [600/600], Loss: 0.1139
Epoch [5/10], Step [600/600], Loss: 0.0560
Epoch [6/10], Step [600/600], Loss: 0.0573
Epoch [7/10], Step [600/600], Loss: 0.0255
Epoch [8/10], Step [600/600], Loss: 0.0272
Epoch [9/10], Step [600/600], Loss: 0.0714
Epoch [10/10], Step [600/600], Loss: 0.0249


In [23]:
# Test the model
# No gradients for the test phase 
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        labels = labels
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs.data, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')

Accuracy of the network on the 10000 test images: 97.45 %
